In [3]:
import matplotlib.pyplot as plt
import seaborn as sb
import glob
import os 
import datetime
from requests import get
from os import path, getcwd, makedirs, listdir
from io import BytesIO
from zipfile import ZipFile

%matplotlib inline

In [7]:
#create folder:
folder_data = 'data'

if not os.path.exists(folder_data):
    os.makedirs(folder_data)
else:
    print('Folder already exists')
        
#loop through all files (from url) starting at month 1 to 12:
for month in range(1,13):
    month_string = str(month)
    month_leading_zero = month_string.zfill(2)
    
    #scrape URL:                                                      +01, 02, 03...eg:201801-fordgobike-tripdata.csv.zip
    bike_data_url = 'https://s3.amazonaws.com/fordgobike-data/2018' + month_leading_zero + '-fordgobike-tripdata.csv.zip'
    response = get(bike_data_url)

    # code below opens zip file; BytesIO returns a view of the contents;
    unzipped_file = ZipFile(BytesIO(response.content))

    # place extracted zip files into folder_data
    unzipped_file.extractall(folder_data)


Folder already exists


In [8]:
# Combine all downloaded CSVs into one and importing into pandas 1st
#import pandas as pd

#list_data = []

#for file_name in listdir(folder_data):
#    list_data.append(pd.read_csv(folder_data+'/'+file_name))
#df = pd.concat(list_data, join = 'inner', sort=True)



#--------------------------------------------------------------------------------------------------------------------#
# only run this block if you want the data to be imported into a pd df first before the vaex conversion           #
#--------------------------------------------------------------------------------------------------------------------#

### Import Vaex lib and convert all downloaded csv files to HDF5 format:
- **convert to HDF5 because vaex support this format**

In [ ]:
#import lib:
import vaex

In [39]:
#check all files are in dir:
os.listdir()

In [29]:
#current working dir?
os.getcwd()

'C:\\Users\\hp\\Downloads\\VAEX example project'

In [32]:
#change cwd to dir that has all csv files:
os.chdir("C:\\Users\\hp\\Downloads\\VAEX example project\\data")

In [33]:
os.getcwd()

'C:\\Users\\hp\\Downloads\\VAEX example project\\data'

In [37]:
# Convert each CSV file to HDF5:
for file in os.listdir(): #make sure to cwd to folder where data is stored for this code to work
    df_tmp = vaex.from_csv(file, convert=True, copy_index=False)

In [40]:
#take all converted files(HDF5 files) and open with vaex in one df:
df = vaex.open('C:\\Users\\hp\\Downloads\\VAEX example project\\data\\*.csv.hdf5')

### Assess:

In [41]:
df

#,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip
0,75284,'2018-01-31 22:52:35.2390','2018-02-01 19:47:19.8240',120.0,'Mission Dolores Park',37.7614205,-122.4264353,285.0,"""Webster St at O'Farrell St""",37.78352083526095,-122.43115782737732,2765,'Subscriber',1986.0,'Male','No'
1,85422,'2018-01-31 16:13:34.3510','2018-02-01 15:57:17.3100',15.0,"""'San Francisco Ferry Building (Harry Bridges Pl...",37.795392,-122.394203,15.0,"""'San Francisco Ferry Building (Harry Bridges Pl...",37.795392,-122.394203,2815,'Customer',nan,--,'No'
2,71576,'2018-01-31 14:23:55.8890','2018-02-01 10:16:52.1160',304.0,'Jackson St at 5th St',37.348758686744794,-121.89479783177376,296.0,'5th St at Virginia St',37.3259984,-121.87711999999999,3039,'Customer',1996.0,'Male','No'
3,61076,'2018-01-31 14:53:23.5620','2018-02-01 07:51:20.5000',75.0,'Market St at Franklin St',37.773793206088705,-122.42123901844023,47.0,'4th St at Harrison St',37.78095459960753,-122.39974915981293,321,'Customer',nan,--,'No'
4,39966,'2018-01-31 19:52:24.6670','2018-02-01 06:58:31.0530',74.0,'Laguna St at Hayes St',37.77643481920474,-122.42624402046205,19.0,'Post St at Kearny St',37.788975,-122.403452,617,'Subscriber',1991.0,'Male','No'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"1,863,716",473,'2018-12-01 00:11:54.8110','2018-12-01 00:19:48.5470',345.0,'Hubbell St at 16th St',37.76647421515012,-122.39829540252686,81.0,'Berry St at 4th St',37.77588,-122.39317,3035,'Subscriber',1982.0,'Female','No'
"1,863,717",841,'2018-12-01 00:02:48.7260','2018-12-01 00:16:49.7660',10.0,'Washington St at Kearny St',37.795392937254526,-122.4047702550888,58.0,'Market St at 10th St',37.776619000000004,-122.41738500000001,2034,'Subscriber',1999.0,'Female','No'
"1,863,718",260,'2018-12-01 00:05:27.6150','2018-12-01 00:09:47.9560',245.0,'Downtown Berkeley BART',37.870139,-122.268422,255.0,'Virginia St at Shattuck Ave',37.876572549106854,-122.26952791213988,2243,'Subscriber',1991.0,'Male','No'
"1,863,719",292,'2018-12-01 00:03:06.5490','2018-12-01 00:07:59.0800',93.0,'4th St at Mission Bay Blvd S',37.770407399999996,-122.39119840000002,126.0,'Esprit Park',37.7616343,-122.3906477,545,'Subscriber',1963.0,'Male','No'


In [45]:
#no of rows and columns:
df.shape

(1863721, 16)

In [47]:
#summary stats:
df.describe().T

,data_type,count,NA,mean,std,min,max
duration_sec,int64,1863721,0,857.3025710393348,2370.38,61,86366
start_time,string,1863721,0,--,--,--,--
end_time,string,1863721,0,--,--,--,--
start_station_id,float64,1851950,11771,119.67443613488486,100.398,3,381
start_station_name,string,1851950,11771,--,--,--,--
start_station_latitude,float64,1863721,0,37.766780024614675,0.105769,37.2633,45.51
start_station_longitude,float64,1863721,0,-122.34918427194773,0.165463,-122.474,-73.57
end_station_id,float64,1851950,11771,118.17297605226923,100.44,3,381
end_station_name,string,1851950,11771,--,--,--,--
end_station_latitude,float64,1863721,0,37.76690033448883,0.105648,37.2633,45.51


In [52]:
df.info()

column,type,unit,description,expression
duration_sec,int64,,,
start_time,str,,,
end_time,str,,,
start_station_id,float64,,,
start_station_name,str,,,
start_station_latitude,float64,,,
start_station_longitude,float64,,,
end_station_id,float64,,,
end_station_name,str,,,
end_station_latitude,float64,,,


### Clean data:

In [70]:
#change incorrect date columns to date time format: 

#start_time:
df['start_time'] = df['start_time'].astype('datetime64[ns]') #vaex doesn't recognize 'datetime64'

#end_time:
df['end_time'] = df['end_time'].astype('datetime64[ns]')

In [73]:
#test:
df['end_time'].data_type(), df['start_time'].data_type()

(datetime64[ns], datetime64[ns])

In [102]:
#create column for user age:
df['member_age'] = 2018 - df['member_birth_year'].astype('int')

In [91]:
#test:
df.member_age.value_counts()

-2147481630    110718
 30            102836
 31             86554
 29             84830
 28             81999
                ...  
 109                4
 129                3
 112                2
 115                1
 88                 1
Length: 87, dtype: int64

In [77]:
df[df.member_age == -2147481630][['member_birth_year', 'member_age']]

#,member_birth_year,member_age
0,nan,-2147481630
1,nan,-2147481630
2,nan,-2147481630
3,nan,-2147481630
4,nan,-2147481630
...,...,...
"110,713",nan,-2147481630
"110,714",nan,-2147481630
"110,715",nan,-2147481630
"110,716",nan,-2147481630


In [92]:
df[df.member_age == -2147481630].count()

array(110718, dtype=int64)

In [105]:
#change strange values above to 0:
import numpy as np

df['member_age_new'] = df.func.where(df['member_age'] == -2147481630, 0, df['member_age'])

In [106]:
df.head(2)

#,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip,member_age,member_age_new
0,75284,2018-01-31 22:52:35.239000000,2018-02-01 19:47:19.824000000,120,'Mission Dolores Park',37.7614,-122.426,285,"""Webster St at O'Farrell St""",37.7835,-122.431,2765,'Subscriber',1986,'Male','No',32,32
1,85422,2018-01-31 16:13:34.351000000,2018-02-01 15:57:17.310000000,15,"""'San Francisco Ferry Building (Harry Bridges Pl...",37.7954,-122.394,15,"""'San Francisco Ferry Building (Harry Bridges Pl...",37.7954,-122.394,2815,'Customer',nan,--,'No',-2.14748e+09,0


In [107]:
#delete old col:
del(df.member_age)

In [112]:
#create days of the week col:


#import datetime lib:
import datetime as dt

df['weekday'] = df['start_time'].dt.strftime('%a')

In [113]:
#test:
df.head(2)

#,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip,member_age,member_age_new,weekday
0,75284,2018-01-31 22:52:35.239000000,2018-02-01 19:47:19.824000000,120,'Mission Dolores Park',37.7614,-122.426,285,"""Webster St at O'Farrell St""",37.7835,-122.431,2765,'Subscriber',1986,'Male','No',32,32,'Wed'
1,85422,2018-01-31 16:13:34.351000000,2018-02-01 15:57:17.310000000,15,"""'San Francisco Ferry Building (Harry Bridges Pl...",37.7954,-122.394,15,"""'San Francisco Ferry Building (Harry Bridges Pl...",37.7954,-122.394,2815,'Customer',nan,--,'No',-2.14748e+09,0,'Wed'


In [115]:
#creaet month col:
df['month'] = df['start_time'].dt.strftime('%B')

In [133]:
#test:
df.column_names

['duration_sec',
 'start_time',
 '____start_time_1',
 '__start_time',
 'end_time',
 '__end_time_1',
 '__end_time',
 'start_station_id',
 'start_station_name',
 'start_station_latitude',
 'start_station_longitude',
 'end_station_id',
 'end_station_name',
 'end_station_latitude',
 'end_station_longitude',
 'bike_id',
 'user_type',
 'member_birth_year',
 'member_gender',
 'bike_share_for_all_trip',
 '__member_age',
 '__member_age_1',
 '__member_age_2',
 '__member_age_new',
 'member_age',
 'member_age_new',
 'weekday',
 'month']

In [117]:
df['month'].value_counts()

October      201458
July         199222
June         195968
August       192162
September    186217
May          179125
November     134135
December     131363
April        131169
March        111382
February     106718
January       94802
dtype: int64

In [140]:
#age outliers:
df.describe()

NameError: name '__start_time_1' is not defined

In [145]:
del('__start_time_1')

SyntaxError: can't delete literal (<ipython-input-145-ae24835029bb>, line 1)

In [144]:
df.member_age_new.min(), df.member_age_new.max()

(array(0), array(137))